In [ ]:
try:
    import transformers
except ImportError:
    !pip install transformers
    import transformers
try:
    import emoji
except ImportError:
    !pip install emoji
    import emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 5.4 MB/s 
     |████████████████████████████████| 6.6 MB 19.1 MB/s 
     |████████████████████████████████| 101 kB 6.7 MB/s 
     |████████████████████████████████| 596 kB 62.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 197 kB 6.0 MB/s 
  Created wheel for emoji: filename=emoji-2.0.0-py3-none-any.whl size=193022 sha256=8d4dafa57e0208ba69df2394a424955a1787a549e490a7a7c3168bfd905d3500
  Stored in directory: /root/.cache/pip/wheels/ec/29/4d/3cfe7452ac7d8d83b1930f8a6205c3c9649b24e80f9029fc38
Successfully built emoji


In [ ]:
local_run = False
local_path = "C:\\Greenwich\\MSc Project\\project_code\\"
store_local = False

In [ ]:
import pandas as pd
import numpy as np

import random
import string
import re
import ast

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

if local_run:
    nltk.data.path.append('/Users/algin/VOLD/nltk_data')
    nltk.download('stopwords',download_dir='/Users/algin/VOLD/nltk_data')
    nltk.download('punkt',download_dir='/Users/algin/VOLD/nltk_data')
else:
    nltk.download('stopwords')
    nltk.download('punkt')
stop_words = set(stopwords.words('english'))

from sklearn.model_selection import train_test_split
import tensorflow as tf
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TFAutoModelForSequenceClassification
from transformers import TFDistilBertForSequenceClassification, TrainingArguments, Trainer
from transformers import DataCollatorWithPadding

from sklearn.metrics import confusion_matrix,f1_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
full_train_data = pd.read_csv("https://github.com/Voldegin/hate_speech_detection/blob/develop/data/mixed/mixed_train_data.csv?raw=true")
test_data = pd.read_csv("https://github.com/Voldegin/hate_speech_detection/blob/develop/data/mixed/mixed_test_data.csv?raw=true")

In [ ]:
train_data, val_data = train_test_split(full_train_data,test_size=5000,random_state=21)

In [ ]:
len(train_data), len(val_data), len(test_data)

(84979, 5000, 10000)

**Splitting into labels and features**

In [ ]:
train_data.head()

,tweet_text,is_cyberbullying,cleaned
51990,"""\n Yes, it is. But a demo is not notable enou...",0,ye is demo notabl enough headlin alreadi writt...
5095,Has it occurred to you that I wouldn't tell so...,1,occur tell someon fuck first refer edit bullsh...
64041,Notice of Edit warring noticeboard discussion\...,0,notic edit war noticeboard discuss hello messa...
53391,""":It doesn't even seem that an amendment is su...",0,it even seem amend subject p5 vote perman five...
10,"""\n\nUNBLOCK ME OR I'LL GET MY LAWYERS ON TO Y...",1,unblock ill get lawyer block constitut right f...


In [ ]:
def get_emoji_regexp():
    # Sort emoji by length to make sure multi-character emojis are
    # matched first
    emojis = sorted(emoji.EMOJI_DATA, key=len, reverse=True)
    pattern = u'(' + u'|'.join(re.escape(u) for u in emojis) + u')'
    return re.compile(pattern)

In [ ]:
banned_list= string.punctuation
punctuation_reg_exp = "[" + banned_list + "]"
emoji_reg_exp = get_emoji_regexp()

def clean_text(text):
    text = text.replace('\r', '').replace('\n', ' ').lower()
    text = re.sub(r"(?:\@|https?\://)\S+", "", text)

    text = [word for word in text.split() if word not in stop_words]
    text = ' '.join(text)

    text = " ".join(word.strip() for word in re.split('#(?!(?:hashtag)\b)[\w-]+(?=(?:\s+#[\w-]+)*\s*$)', text))
    
    text = re.sub(punctuation_reg_exp,"",text)

    text = re.sub("\s\s+" , " ", text)

    text = re.sub(emoji_reg_exp, r"", text)

    return text

In [ ]:
def split_label_and_feature(data):
    data['cleaned'] = data['tweet_text'].apply(clean_text)
    return data['cleaned'], data['is_cyberbullying']

In [ ]:
X_train, y_train = split_label_and_feature(train_data)
X_val, y_val = split_label_and_feature(val_data)
X_test, y_test = split_label_and_feature(test_data)

In [ ]:
X_train.head()

51990     yes is demo notable enough headline already w...
5095     occurred tell someone fuck first referred edit...
64041    notice edit warring noticeboard discussion hel...
53391    it even seem amendment subject p5 vote permane...
10        unblock ill get lawyers blocking constitution...
Name: cleaned, dtype: object

**Model Training**

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

MODEL_NAME = 'distilbert-base-uncased'
model = TFDistilBertForSequenceClassification.from_pretrained(MODEL_NAME)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_layer_norm', 'vocab_projector', 'activation_13', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'dropout_19', 'pre_classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [ ]:
def create_tf_dataset(X_feature,X_label):
    encodings = tokenizer(list(X_feature.values),truncation=True,padding=True)
    dataset = tf.data.Dataset.from_tensor_slices((dict(encodings),list(X_label.values)))
    return dataset

In [ ]:
train_dataset = create_tf_dataset(X_train, y_train)

In [ ]:
val_datset = create_tf_dataset(X_val, y_val)

In [ ]:
# for _layer in model.layers:
#     if _layer.name == 'distilbert':
#         print(f"Freezing model layer {_layer.name}")
#         _layer.trainable = False
#     print(_layer.name)
#     print(_layer.trainable)
model.summary()

Model: "tf_distil_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMai  multiple                 66362880  
 nLayer)                                                         
                                                                 
 pre_classifier (Dense)      multiple                  590592    
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
 dropout_19 (Dropout)        multiple                  0         
                                                                 
Total params: 66,955,010
Trainable params: 66,955,010
Non-trainable params: 0
_________________________________________________________________


In [ ]:
BATCH_SIZE = 16
N_EPOCHS = 3

#chose the optimizer
optimizerr = tf.keras.optimizers.Adam(learning_rate=5e-5)
#define the loss function 
losss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
#build the model
model.compile(optimizer=optimizerr,
              loss=losss,
              metrics=['accuracy'])
# train the model 
model.fit(train_dataset.shuffle(len(X_train)).batch(BATCH_SIZE),
          epochs=N_EPOCHS,
          batch_size=BATCH_SIZE)

Epoch 1/3
5312/5312 [==============================] - 2986s 560ms/step - loss: 0.2438 - accuracy: 0.8968
Epoch 2/3
5312/5312 [==============================] - 2974s 560ms/step - loss: 0.1734 - accuracy: 0.9271
Epoch 3/3
5312/5312 [==============================] - 2973s 560ms/step - loss: 0.1162 - accuracy: 0.9533


**Saving Model**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/MyDrive/MSc\ Data\ Science/MSc\ Project/models

[Errno 2] No such file or directory: 'drive/MyDrive/MSc Data Science/MSc Project/models'
/content


In [ ]:
model.save_pretrained('mixed-distilbert--without-stem-94')

In [ ]:
def predict_proba(text_list, model, tokenizer):  
    #tokenize the text
    encodings = tokenizer(text_list, 
                          truncation=True, 
                          padding=True)
    #transform to tf.Dataset
    dataset = tf.data.Dataset.from_tensor_slices((dict(encodings)))
    #predict
    preds = model.predict(dataset.batch(1)).logits  
    
    #transform to array with probabilities
    res = tf.nn.softmax(preds, axis=1).numpy()      
    
    return res

In [ ]:
predict_proba(["What a good day"],model,tokenizer)

array([[0.98569655, 0.0143034 ]], dtype=float32)

In [ ]:
predict_proba(["Muslims are terrorists"],model,tokenizer)

array([[0.00184221, 0.9981578 ]], dtype=float32)

In [ ]:
predict_proba(["You are an asshole"],model,tokenizer)

array([[0.00405474, 0.9959453 ]], dtype=float32)

In [ ]:
predict_proba(["I believe in Christianity"],model,tokenizer)

array([[0.9612582 , 0.03874181]], dtype=float32)